In [1]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_ollama import ChatOllama
from langchain_core.runnables import RunnableLambda


# 데이터 모델 클래스
class RouteQuery(BaseModel):
    datasource: Literal["python_docs", "js_docs"] = Field(
        ...,
        description="Given a user question, choose which datasource would be most relevant for answering their question",
    )

/home/anseh/.pyenv/versions/langchain/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 프롬프트 템플릿
# 함수 호출
llm = ChatOllama(model="gemma3:1b", temperature=0)

"""
with_structured_output: 주어진 스키마와 일치하도록 형식화된 출력을 반환하는 모델 래퍼

"""
structured_llm = llm.with_structured_output(RouteQuery)

# 프롬프트
system = """당신은 사용자 질문을 적절한 데이터 소스로 라우팅하는 전문가입니다. 질문이 지목하는 프로그래밍 언어에 따라 해당 데이터 소스로 라우팅하세요."""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}")])

# 라우터 정의
router = prompt | structured_llm

In [3]:
# 실행
question = """이 코드가 안 돌아가는 이유를 설명해주세요: 
from langchain_core.prompts 
import ChatPromptTemplate 
prompt = ChatPromptTemplate.from_messages(['human', 'speak in {language}']) 
prompt.invoke('french') """

result = router.invoke({"question": question})
print("\nRouting to: ", result)


def choose_route(result):
    if "python_docs" in result.datasource.lower():
        return "chain for python_docs"
    else:
        return "chain for js_docs"


full_chain = router | RunnableLambda(choose_route)

result = full_chain.invoke({"question": question})
print("\nChoose route: ", result)


Routing to:  datasource='python_docs'

Choose route:  chain for python_docs


### 의미론적 라우팅

In [ ]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import chain
from langchain_ollama import ChatOllama, OllamaEmbeddings

physics_template = """당신은 매우 똑똑한 물리학 교수입니다. 
    당신은 물리학에 대한 질문에 간결하고 쉽게 이해할 수 있는 방식으로 대답하는 데 뛰어납니다.
    당신이 질문에 대한 답을 모를 때는 모른다고 인정합니다.
    다음 질문에 답하세요.: {query}
    """
math_template = """
    당신은 매우 뛰어난 수학자입니다. 당신은 수학 문제에 답하는 데 뛰어납니다.
    당신은 어려운 문제를 구성 요소로 분해하고 구성 요소를 해결한 다음
    함께 모아 더 넓은 질문에 대답합니다.
    다음 질문에 답하세요.: {query}
    """

# 임베딩
embeddings = OllamaEmbeddings(model="nomic-embed-text")
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

In [ ]:
@chain
def prompt_router(query):
    query_embedding = embeddings.embed_query(query)
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    print(
        "수학 프롬프트 사용" if most_similar == math_template else "물리 프롬프트 사용"
    )
    return PromptTemplate.from_template(most_similar)


semantic_router = prompt_router | ChatOllama(model="gemma3:1b") | StrOutputParser()

result = semantic_router.invoke("블랙홀이란 무엇인가요?")
print("\n의미론적 라우팅 결과: ", result)